<h1 style="text-align: center">
Deep Learning HW4 </br>
Poem generation,fine tuning
</h1>



  <style>
    table {
      width: 100%;
      border-collapse: collapse;
    }

    th, td {
      border: 1px solid #dddddd;
      text-align: left;
      padding: 8px;
    }

    th {
      background-color: #f2f2f2;
    }

    .box {
      border: 1px solid #000;
      padding: 10px;
      width: 400px; /* Adjust the width as needed */
      margin: 20px auto;
    }
  </style>
</head>
<body>

<div class="box">
  <table>
    <tr>
      <th colspan="2">Personal Info</th>
    </tr>
    <tr>
      <td>First Name:</td>
      <td>Ali</td>
    </tr>
    <tr>
      <td>Last Name:</td>
      <td>Nikkhah</td>
    </tr>
    <tr>
      <td>Student Number:</td>
      <td>99102445</td>
    </tr>
    <tr>
      <td>Git:</td>
      <td><a href="https://github.com/AliNikkhah2001/DataScience02" target="_blank">https://github.com/AliNikkhah2001/DeepLearning02</a></td>
    </tr>
  </table>
</div>



# Poem generator using Ferdowsi poems dataset

## Data Import and initialization

In [2]:
%%capture
!pip install keras_preprocessing
!pip install transformers


In [3]:
%%capture
import tensorflow as tf
import numpy as np
import math
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [4]:

from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config
from IPython import display
     

## Open text file and make dataframe 

In [5]:
!git lfs install
!git clone https://huggingface.co/HooshvareLab/gpt2-fa-poetry

Git LFS initialized.
fatal: destination path 'gpt2-fa-poetry' already exists and is not an empty directory.


In [6]:
data_path='/kaggle/input/ferdowsi/ferdousi.txt'

In [7]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
lines = lines[2:]


In [8]:
model_name_or_path = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    bos_token='<s>', 
    eos_token='</s>', 
    pad_token='<pad>',
    unk_token='<unk>'
)
tokenizer.add_special_tokens({
    "bos_token": '</s>',
    "eos_token": '</s>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

config = AutoConfig.from_pretrained(
    model_name_or_path,
    bos_token_id=tokenizer("<s>")["input_ids"][0], 
    eos_token_id=tokenizer("</s>")["input_ids"][0], 
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)

tokenizer.save_pretrained("/kaggle/working/gpt2-fa/")
config.save_pretrained("/kaggle/working/gpt2-fa/")

In [9]:
df_input=[]
df_target=[]
df_concat=[]
for i in range(0, len(lines)-3, 2):
    input_text= lines[i] 
    target_text = ' <s> '+ lines[i+1]+'   ' +lines[i+2] + '    ' + lines[i+3]+'  </s>  '
    df_input.append(input_text)
    df_target.append(target_text)
    df_concat.append(input_text+target_text)

In [10]:
print(len(df_input))
print(len(df_target))

49607
49607


In [11]:
idx = 50
print(f'{df_input[idx]}')
print(f'{df_target[idx]}')
print(f'{df_concat[idx]}')

همی بر شد آتش فرود آمد آب
 <s> همی گشت گرد زمین آفتاب   گیا رست با چند گونه درخت    به زیر اندر آمد سرانشان ز بخت  </s>  
همی بر شد آتش فرود آمد آب <s> همی گشت گرد زمین آفتاب   گیا رست با چند گونه درخت    به زیر اندر آمد سرانشان ز بخت  </s>  


In [12]:

print(tokenizer.encode("<s>گیا رست با چند گونه درخت    به زیر اندر آمد سرانشان ز بخت</s>"))
print(tokenizer.encode("<s>"))
print(tokenizer.encode("</s>"))
print(tokenizer.encode("<pad>"))
print(tokenizer.encode("<|startoftext|>"))
print(tokenizer.encode("<sep>"))

[0, 423, 268, 2630, 314, 638, 1752, 3087, 21147, 303, 730, 15233, 2199, 3325, 549, 355, 11653, 2]
[0]
[2]
[1]
[6]
[9]


## MultiGPU setup

In [13]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Jan 11 09:45:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
import torch

def set_up_two_gpus():
    available_gpus = torch.cuda.device_count()
    if available_gpus < 2:
        raise ValueError("Insufficient number of GPUs available. Need at least two GPUs.")

    device_1 = torch.device("cuda:0")
    device_2 = torch.device("cuda:1")

    print(f"Available GPUs: {available_gpus}")
    print(f"Setting GPU 0 as device: {device_1}")
    print(f"Setting GPU 1 as device: {device_2}")

    return device_1, device_2

# Check and print if the GPUs are working by performing a computational operation
def check_gpu(device):
    tensor = torch.randn(1000, 1000, device=device)
    result = torch.matmul(tensor, tensor)
    return result

# Usage
device_1, device_2 = set_up_two_gpus()
result_1 = check_gpu(device_1)
result_2 = check_gpu(device_2)
print("Results from GPU 0:", result_1)
print("Results from GPU 1:", result_2)


Available GPUs: 2
Setting GPU 0 as device: cuda:0
Setting GPU 1 as device: cuda:1
Results from GPU 0: tensor([[ 15.7754,  28.1131,  19.3813,  ...,   2.1333, -21.8276, -18.7964],
        [ 21.8896, -23.0110,  42.5994,  ...,  -2.9641,  12.9708, -14.1444],
        [-21.1606,   1.9071,  -1.7755,  ...,  21.4530, -31.3334,   6.3431],
        ...,
        [-34.3632,  -2.3821,  -7.7075,  ...,  11.2810, -90.0004, -10.4166],
        [-32.6388, -29.5234, -15.5900,  ..., -30.7012,  -5.7184, -68.0117],
        [ 28.7030, -25.0085, -13.7762,  ..., -22.9083,  28.3056, -52.3145]],
       device='cuda:0')
Results from GPU 1: tensor([[-48.7228, -59.0386, -28.8195,  ...,  22.8311,  24.0622,  35.2678],
        [ 29.6720,  12.5468, -67.6738,  ...,   4.0252,  48.0882,  80.3356],
        [-20.7171, -42.0483,  69.3719,  ...,  38.2693,  45.1786,  42.2385],
        ...,
        [ 13.9805, -24.3852,  43.2172,  ...,   9.4317,  -4.9429, -48.4698],
        [ 11.4831,  -0.7422,   2.8613,  ..., -27.0245,  14.7413, -5

In [15]:
# Sample input text
input_text = df_input[50]
# Tokenize and encode the input text
input_encoding = tokenizer.encode(input_text)

# Decode the tokenized input back to text
decoded_input = tokenizer.decode(input_encoding)

# Print results
print("Original Text:", input_text)
print("Tokenized Input:", input_encoding)
print("Decoded Input:", decoded_input)


Original Text: همی بر شد آتش فرود آمد آب
Tokenized Input: [272, 398, 327, 403, 2466, 2638, 2199, 797]
Decoded Input: همی بر شد آتش فرود آمد آب


In [16]:
from torch.utils.data import Dataset  
import torch
torch.manual_seed(42)
class MTGDataset(Dataset):
    def __init__(self, input_texts, tokenizer, max_length=1024):
        self.tokenizer = tokenizer  
        self.input_ids = []
        self.attn_masks = []

        for txt in input_texts:
            encodings_dict = tokenizer(txt ,
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [17]:
from torch.utils.data import random_split
max_seq = 256
dataset = MTGDataset(df_concat, tokenizer, max_length=max_seq)
# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = (len(dataset) - train_size)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {len(train_dataset)} samples for training, and {len(val_dataset)} samples for validation testing'

'There are 44646 samples for training, and 4961 samples for validation testing'

In [18]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('/kaggle/working/gpt2-fa/', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("/kaggle/working/gpt2-fa/", config=configuration)
model.resize_token_embeddings(len(tokenizer))


# This step is optional but will enable reproducible runs.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [19]:
epochs = 3
warmup_steps = 1e2
sample_every = 300

In [20]:
from transformers import AdamW

# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters.
optimizer = AdamW(
    model.parameters(),
    lr=5e-4,
    eps=1e-8,
    no_deprecation_warning=True
)

In [21]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=8
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=8
)

In [22]:
from transformers import get_linear_schedule_with_warmup
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps)


In [23]:
from torch.nn import DataParallel
# Define a function to set up multi-GPU training
def multi_gpu_setup(model):
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs")
        model = DataParallel(model)
    return model

# Use the function to set up multi-GPU training
model = multi_gpu_setup(model)

Using 2 GPUs


In [24]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cuda':
    print(f"Memory Allocated: {torch.cuda.memory_allocated(device)/1024**3:.2f} GB")
    print(f"Memory Cached: {torch.cuda.memory_cached(device)/1024**3:.2f} GB")
else:
    print("CUDA is not available. Switching to CPU.")


Using device: cuda
Memory Allocated: 0.01 GB
Memory Cached: 0.02 GB


/opt/conda/lib/python3.10/site-packages/torch/cuda/memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [25]:

sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to('cpu')

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

جوان گرچه دانا بود با گهر
Example output: جوان گرچه دانا بود با گهربار و با فراستش این کار را کرد و مردم را برانداخت به تباهی. پس از او، نوبت به یزدگرد رسید. سپاه به سوی غرب رفت. سپاه عرب در آن سوی جیحون (که در زبان فارسی از
Example output: جوان گرچه دانا بود با گهربار عشق» اما در این میان که چه چیزی از قلم افتاده است. در این میان می‌بینم «ای مردم، چرا نباید در دل آتش به گریه بیفتد؟» تا این که یک نفر از مردم «هست
Example output: جوان گرچه دانا بود با گهربار که این هم یک بار خیلی هم آدم را خسته نکرد و از فرط خستگی به خودش آمد. او را با این حال صدا می‌کند، می‌گوید: «با همین حال می‌توانم به خودم امید
Example output: جوان گرچه دانا بود با گهربار در کوچه پس‌کوچه‌ها قدم زد. با صدای آواز دلکش و نغمه‌هایی دلنشین از بنان. به آواز روح می‌افروزم و آرام می‌نشیند. می‌آید و می‌نشیند
Example output: جوان گرچه دانا بود با گهربار و با خود می‌گوید من هنوز به اندازهٔ تو برای زندگی کردن احتیاج دارم (پس من هم به همین نسبت احتیاج دارم) و فقط وقتی بزرگ شدم این مشکل‌ها را برطرف و از آن‌ها


In [26]:
%%capture
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
torch.cuda.empty_cache()

In [32]:
from tqdm import tqdm


In [29]:

training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    learning_rate=5e-5,
    logging_steps=500,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=1000,
    # Add distributed training options
    per_device_eval_batch_size=32,
    fp16=True,  # Enable mixed-precision training if supported
    dataloader_num_workers=16,  # Adjust based on your system capabilities
    report_to="tensorboard",  # You can use TensorBoard for logging
)

# Initialize Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)




In [33]:
import random
import time
import datetime
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))
total_t0 = time.time()
training_stats = []
model = model.to(device)
for epoch_i in tqdm(range(training_args.num_train_epochs),position=0):
    print(f'Beginning epoch {epoch_i + 1} of {training_args.num_train_epochs}')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        model.zero_grad()
        outputs = model(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)
        loss = outputs.loss
        loss = loss.mean()
        batch_loss = loss
        total_train_loss += batch_loss
        # Get sample every 100 batches.
        if step % sample_every == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print()
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')
            model.eval()
            sample_input = df_input[np.random.randint(0, len(df_input))]
            print(sample_input)
            sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
            sample_input_ids = sample_input_ids.to(device)
            sample_outputs = model.module.generate(
                input_ids=sample_input_ids,
                do_sample=True,
                top_k=50,
                max_length=50,
                top_p=0.95,
                num_return_sequences=1
            )
            for i, sample_output in enumerate(sample_outputs):
                gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
                gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
                gen_sample_output = gen_sample_output.replace("<s>", "")
                gen_sample_output = gen_sample_output.replace("</s>", "")
                gen_sample_output = gen_sample_output.replace("<sep>", "\n")
                print(f'Example output: {gen_sample_output}')
            model.train()
        loss.backward()
        optimizer.step()
        scheduler.step()
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print()
    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')
    print()
    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_masks, labels=b_labels)
            loss = outputs.loss
            loss = loss.mean()
            batch_loss = loss
        total_eval_loss += batch_loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    print()
    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')
    print()
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')

  0%|          | 0/1 [00:00<?, ?it/s]

Beginning epoch 1 of 1


  5%|▌         | 300/5581 [02:06<37:04,  2.37it/s]


Batch 300 of 5581. Loss:0.5932118892669678. Time:0:02:07
جوان گرچه دانا بود با گهر
Example output: جوان گرچه دانا بود با گهرنگار  بران کار توانا بود از گوهر و گهر    نه گوهر بود اندر بر تو به کار    وگر نه گنج یابد تو را  


 11%|█         | 600/5581 [04:13<34:57,  2.37it/s]

...


Batch 5400 of 5581. Loss:0.3219226002693176. Time:0:38:04
مبادا ز تو تخت پردخت و گاه
Example output: مبادا ز تو تخت پردخت و گاه  اگر داد بیند تو باشی سزاوار شاه   نگر تا ز فرمان من شاه باز    به فرجام نیک و بد بر تو گردن فراز  


100%|██████████| 5581/5581 [39:20<00:00,  2.36it/s]



Average Training Loss: 0.4587778151035309. Epoch time: 0:39:21



100%|██████████| 1/1 [40:57<00:00, 2457.68s/it]


Validation loss: 0.3717266023159027. Validation Time: 0:01:37

Total training took 0:40:58


In [34]:
save_path = "trained.pth"  
torch.save(model.state_dict(), save_path)
print(f'Model saved at: {save_path}')

Model saved at: trained.pth


# Model Generated Poem Samples

In [50]:

sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

به شادی و انده نگردد دگر
Example output: به شادی و انده نگردد دگر  نگونسار گردد همی خون چکاند   کسی کو بود در جهان زنده وار    به فرجام روزیش بر کشته کار  <pad><pad>
Example output: به شادی و انده نگردد دگر  هم از گوهر شاه بی سر و هم فرو   چنین گفت رستم به نزد پسر    که ای شاه دانش پذیر و پاک دربدر  
Example output: به شادی و انده نگردد دگر  چو روشن ببیند به خورشید سر   ز رستم فراوان بپرسید و گفت    که گر یک سوار آمدست اندر نهفت  <pad><pad><pad>
Example output: به شادی و انده نگردد دگر  ز شاهان کند روز سر   به داد و به فر و خرد رهنمای    به فر و گهر باشد اندر سرای  <pad><pad><pad>
Example output: به شادی و انده نگردد دگر  یکی را نشاید سپردن گذر   چنین گفت کای شاه پیروزبخت    به داد و به دانش توانا و تخت  <pad><pad><pad><pad>


In [51]:

sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')
sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

نشسته برو شهریاری چو ماه
Example output: نشسته برو شهریاری چو ماه  به سر بر نهاده کلاه و سپاه   چو برزد سر از کوه خورشید زرد    ز دریا همی کرد تا بود روز  <pad><pad><pad>
Example output: نشسته برو شهریاری چو ماه  به دیهیم زرین به سر تا به ماه   همی راند تا شهر توران سپاه    یکی بندهٔ کردگار کینه خواه  <pad><pad><pad>
Example output: نشسته برو شهریاری چو ماه  نشسته برو بر همه بندگان پیشگاه   چو بشنید ازو آن سخن رهنمای    پس آنگه ز شاه جهان کدخدای  <pad><pad>
Example output: نشسته برو شهریاری چو ماه  که بودی به فرمان و فرمان شاه   هرآنکس که او بود بی او به جنگ    همه رنج و اندوه او بود و تنگ  
Example output: نشسته برو شهریاری چو ماه  به نزد ز اختر یکی شاه شاه و شاه   به پیش من آر ای نامور پهلوان    که شادان بدی تا بود پهلوان  <pad><pad>


In [52]:

sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

بباید بریدن سر خویش پست
Example output: بباید بریدن سر خویش پست  برستم چه باید کشیدن به دست   به گرد جهان من آن کی شوم    ازو چند هرگز نجویم بکوشم ره  
Example output: بباید بریدن سر خویش پست  چو آمدش رای خسرو بدست   بر آنکس که خواند چنین کی قباد    نزادند یک روزگار قباد  <pad><pad><pad><pad><pad>
Example output: بباید بریدن سر خویش پست  وزین نیز چندی دل برفشاند   چو این کرده باشی تو کردار من    چنان بدکنش بی گمان جان بدکیش من  <pad>
Example output: بباید بریدن سر خویش پست  نجویند یک سر بریدن سر خویش پست   چو یزدان مرا دید و فرمان گزید    بدان جای بی چاره سر جان گزید  <pad><pad><pad>
Example output: بباید بریدن سر خویش پست  اگر چاره ای هست با بیش پست   نباید کشیدن سر از تخت خویش    به روز نو سرآید بدان جایگاه  <pad><pad><pad><pad><pad>


In [53]:

sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

به نزد بزرگان ایرانیان
Example output: به نزد بزرگان ایرانیان  همی رفت باید به پیش پدر   چو بهرام یل رابخواند درست    که از درد و تیمار او برفشاند نخست  
Example output: به نزد بزرگان ایرانیان  فرستاد یکسر دوال کمر   چنان چون به هر جای فرمان گزید    همه لشکر آراسته برگزید  <pad><pad><pad><pad><pad><pad>
Example output: به نزد بزرگان ایرانیان  به تخت بزرگی به پای میان   به دست چپش بر یکی پهلوان    همی تاختندی بدین پهلوان  <pad><pad><pad><pad><pad>
Example output: به نزد بزرگان ایرانیان  ز شاه آفریدی به زین   به مردی ز شاه جهان آگهی    کجا بود در جهان آگهی  <pad><pad><pad><pad><pad>
Example output: به نزد بزرگان ایرانیان  ببستند با او میان   چو او بر در دژ دژ رسید آگهی    پس او را سر آمد سپه دهی  <pad><pad>


In [54]:

sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

کنون روز دادست بیداد شد
Example output: کنون روز دادست بیداد شد  جهان را همه روز کامروز شد   تو با من چنان چون گمانی مکوش    مرا رنج و سختیست و چندین خروش  
Example output: کنون روز دادست بیداد شد  ز هر گونه ای کرد کردار یاد   همان روز گشتاسپ اندر گذشت    بدیب و به خون و به دل کینه گشت  <pad><pad>
Example output: کنون روز دادست بیداد شد  که از باد روز نبرد   چنین است گیتی پر آواز من    بدین گونه بر کام تو یافتم  <pad><pad><pad><pad><pad><pad><pad>
Example output: کنون روز دادست بیداد شد  جهان زیر فر چرخ و آهنگ شد   تو از من به من چون توانگر شوی    به فرمان شاه جهان بر شوی  <pad><pad>
Example output: کنون روز دادست بیداد شد  کزین بد چه روز بیداد شد   چه در شهر مازندران    که هر ماه رخ شاهوار ماه  <pad><pad><pad><pad><pad><pad>
